In [ ]:
import pandas as pd
import numpy as np
#import geopandas as gpd
import urllib#pour récupérer les données
import bs4#pour rendre lisibles les données
import re

Récupération des députés sur le site de l'Assemblée Nationale

In [ ]:
def Scrap (url):
    req = urllib.request.Request(url)
    html = urllib.request.urlopen(req).read()
    page = bs4.BeautifulSoup(html, "lxml")
    return page

In [ ]:
def ToDf (table) :
    Listgrossiere=pd.read_html(str(table))#on transforme le tableau en liste de dataframes
    listeDf=Listgrossiere[0]#on récupère le dataframe
    return listeDf

In [ ]:
page=Scrap('http://www2.assemblee-nationale.fr/sycomore/resultats/(offset)/'+str(0)+'/(query)/IiBTRUxFQ1QgbV9jb2RlX2RlcHV0ZSwgaWRfYWN0ZXVyX3RyaWJ1biwgbGVnX21heF90cmlidW4sIG5vbSwgbm9tX2FmZmljaGUsIHByZW5vbSwgZGF0ZV9uYWlzLCBkYXRlX2RlY2VzIEZST00gZGVwdXRlIFdIRVJFIDE9MSAgQU5EIG1fY29kZV9kZXB1dGUgSU4gKFNFTEVDVCBtX2NvZGVfZGVwdXRlIEZST00gbWFuZGF0LCBkZXBhcnRlbWVudCAgV0hFUkUgbWFuZGF0Lm1fbnVtX2RlcCA9IGRlcGFydGVtZW50Lm1fbnVtX2RlcCBBTkQgbWFuZGF0Lm1fdHlwZV9tYW5kYXQgPSAxICkgT1JERVIgQlkgZGF0ZV9uYWlzIERFU0Mi')
table = page.find('table')
liste=ToDf(table)
for i in range(32):
    page=Scrap('http://www2.assemblee-nationale.fr/sycomore/resultats/(offset)/'+str((i+1)*500)+'/(query)/IiBTRUxFQ1QgbV9jb2RlX2RlcHV0ZSwgaWRfYWN0ZXVyX3RyaWJ1biwgbGVnX21heF90cmlidW4sIG5vbSwgbm9tX2FmZmljaGUsIHByZW5vbSwgZGF0ZV9uYWlzLCBkYXRlX2RlY2VzIEZST00gZGVwdXRlIFdIRVJFIDE9MSAgQU5EIG1fY29kZV9kZXB1dGUgSU4gKFNFTEVDVCBtX2NvZGVfZGVwdXRlIEZST00gbWFuZGF0LCBkZXBhcnRlbWVudCAgV0hFUkUgbWFuZGF0Lm1fbnVtX2RlcCA9IGRlcGFydGVtZW50Lm1fbnVtX2RlcCBBTkQgbWFuZGF0Lm1fdHlwZV9tYW5kYXQgPSAxICkgT1JERVIgQlkgZGF0ZV9uYWlzIERFU0Mi')
    table = page.find('table')#on récupère le seul tableau de la page qui liste des députés
    listeDf=ToDf(table)
    liste=pd.concat([liste,listeDf])#on récupère le dataframe et on l'ajoute à la liste
    
liste=liste.reset_index()#on recrée un index afin de ne pas s'arrêter à 500 puis recommencer comme dans les pages web
listeDepute=liste[['Nom','Né le']]#on récupère les variables qui nous intéressent  

In [ ]:
listeDepute.head()

On s'intéresse ensuite à la récupération des mandats de chaque député (partie en cours).

In [ ]:
page=Scrap('http://www2.assemblee-nationale.fr/sycomore/fiche/(num_dept)/'+str(5))
listeMandat=page
for i in range(len(listeDepute)-1) :
    page=Scrap('http://www2.assemblee-nationale.fr/sycomore/fiche/(num_dept)/'+str(i+7))
    

In [118]:
page=Scrap('http://www2.assemblee-nationale.fr/sycomore/fiche/(num_dept)/'+str(5))
#Une petite blague c'est glissé dans le site : les mandats de sénateurs sont présent dans le code html mais ils sont "cachés"
#On les enlève donc d'abord
pouf=page.find('div', {'id':"mandats_an"})
depute=pouf.findAll('dl', {'class' : "deputes-liste-attributs sycomore"})


In [119]:
def scrap_un_mandat(mandat):
    #On récupère et lave les titres des colonnes
    columns=mandat.findAll('dt')
    columns=re.sub('<[^<]+?>', '', str(columns))  
    
    #On récupère et lave les titres des lignes
    lines=mandat.findAll('dd')
    lines=re.sub('<[^<]+?>', '', str(lines))
    lines=re.sub('\t', '', str(lines))
    lines=re.sub("\n", '', str(lines))
    line=''
    
    #On met tout dans un dataframe
    columns=columns[1:-1].split(',')
    lines=lines[1:-1].split(',')
    df=pd.DataFrame([lines],columns=columns)
    return df

In [120]:
scrap_un_mandat(depute[1])

,Régime politique,Législature,Mandat,Département,Groupe
0,Cinquième République - Assemblée nationale,VIe législature,Du 19 mars 1978 au 22 mai 1981,Hautes-Pyrénées,Socialiste


In [121]:
#On fusionne les mandats
liste=[]
for mandat in depute :
    df = scrap_un_mandat(mandat)
    if len(liste)==0: #On initialise avec le premier mandat
        liste=df  
    
    else:
        liste=pd.concat([liste,df])

liste

,Régime politique,Législature,Mandat,Département,Groupe
0,Cinquième République - Assemblée nationale,Ve législature,Du 2 avril 1973 au 2 avril 1978,Hautes-Pyrénées,Parti socialiste et radicaux de gauche
0,Cinquième République - Assemblée nationale,VIe législature,Du 19 mars 1978 au 22 mai 1981,Hautes-Pyrénées,Socialiste
0,Cinquième République - Assemblée nationale,VIIe législature,Du 2 juillet 1981 au 23 juillet 1981,Hautes-Pyrénées,Socialiste
